In [ ]:
import pandas as pd
import altair as alt

# Step 1: Load the cleaned dataset
cleaned_data_file = "Crime_Reports_20250109.csv"  # Replace with your actual file path
crime_data = pd.read_csv(cleaned_data_file)

# Step 2: Data Preparation
# Narrative 1: Crime Types by Location
crime_by_location = crime_data.groupby(['Location Type', 'Highest Offense Description']).size().reset_index(name='Count')

# Narrative 2: Clearance Rates by Offense
clearance_by_offense = crime_data.groupby('Highest Offense Description').agg(
    Total_Incidents=('Incident Number', 'size'),
    Cleared_Incidents=('Clearance Status', lambda x: (x == 'C').sum())
).reset_index()
clearance_by_offense['Clearance Rate'] = (
    clearance_by_offense['Cleared_Incidents'] / clearance_by_offense['Total_Incidents']
) * 100

# Narrative 3: Incident Concentrations by District
incident_counts_by_district = crime_data.groupby('Council District').size().reset_index(name='Incident Count')

# Step 3: Create Altair Charts
crime_chart = alt.Chart(crime_by_location).mark_bar().encode(
    x=alt.X('Location Type:N', sort='-y', title='Location Type'),
    y=alt.Y('Count:Q', title='Crime Count'),
    color='Highest Offense Description:N',
    tooltip=['Location Type', 'Highest Offense Description', 'Count']
).properties(
    title='Crime Types by Location',
    width=600,
    height=400
)

top_offenses = clearance_by_offense.nlargest(10, 'Total_Incidents')
clearance_chart = alt.Chart(top_offenses).mark_bar().encode(
    x=alt.X('Highest Offense Description:N', sort='-y', title='Offense Type'),
    y=alt.Y('Clearance Rate:Q', title='Clearance Rate (%)'),
    tooltip=['Highest Offense Description', 'Clearance Rate', 'Total_Incidents']
).properties(
    title='Clearance Rates by Offense Type (Top 10 Offenses)',
    width=600,
    height=400
)

district_chart = alt.Chart(incident_counts_by_district).mark_bar().encode(
    x=alt.X('Council District:N', sort='-y', title='Council District'),
    y=alt.Y('Incident Count:Q', title='Incident Count'),
    tooltip=['Council District', 'Incident Count']
).properties(
    title='Incident Concentrations by Council District',
    width=600,
    height=400
)

# Step 4: Export the charts as JSON
charts = {
    "crime_chart": crime_chart.to_json(),
    "clearance_chart": clearance_chart.to_json(),
    "district_chart": district_chart.to_json()
}

# Step 5: Create the HTML Template
html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Crime Data Dashboard</title>
    <script src="https://cdn.jsdelivr.net/npm/vega@5"></script>
    <script src="https://cdn.jsdelivr.net/npm/vega-lite@5"></script>
    <script src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>
    <h1>Crime Data Dashboard</h1>

    <h2>Crime Types by Location</h2>
    <div id="crime_chart"></div>
    <script>
        vegaEmbed("#crime_chart", {charts['crime_chart']}).catch(console.error);
    </script>

    <h2>Clearance Rates by Offense</h2>
    <div id="clearance_chart"></div>
    <script>
        vegaEmbed("#clearance_chart", {charts['clearance_chart']}).catch(console.error);
    </script>

    <h2>Incident Concentrations by District</h2>
    <div id="district_chart"></div>
    <script>
        vegaEmbed("#district_chart", {charts['district_chart']}).catch(console.error);
    </script>

</body>
</html>
"""

# Step 6: Save the Combined HTML File
with open("combined_crime_dashboard.html", "w") as f:
    f.write(html_template)

print("Combined dashboard saved as 'combined_crime_dashboard.html'. Open this file in your browser to view all visualizations.")


In [ ]:
# Step 1: Load the cleaned dataset
cleaned_data_file = "Crime_Reports_20250109.csv"  # Replace with your actual file path
crime_data = pd.read_csv(cleaned_data_file)

# Step 2: Prepare the data
# Group by incident type and location
crime_by_location = crime_data.groupby(['Location Type', 'Highest Offense Description']).size().reset_index(name='Count')

# Step 3: Create a dropdown selection for incident type
incident_dropdown = alt.binding_select(
    options=list(crime_by_location['Highest Offense Description'].unique()),
    name='Incident Type: '
)
incident_selection = alt.selection_point(
    fields=['Highest Offense Description'],
    bind=incident_dropdown,
    value=list(crime_by_location['Highest Offense Description'].unique())[0]  # Default value must match the dropdown options
)

# Step 4: Filtered chart based on the selection
dropdown_chart = alt.Chart(crime_by_location).mark_bar().encode(
    x=alt.X('Location Type:N', sort='-y', title='Location Type'),
    y=alt.Y('Count:Q', title='Crime Count'),
    tooltip=['Location Type', 'Count', 'Highest Offense Description'],
    color=alt.Color('Location Type:N', legend=None)
).add_selection(
    incident_selection
).transform_filter(
    incident_selection
).properties(
    title='Crime Count by Location Type for Selected Incident Type',
    width=800,
    height=400
)

# Step 5: Save or display the chart
dropdown_chart.save('crime_count_by_incident_dropdown.html')
print("Visualization saved as 'crime_count_by_incident_dropdown.html'. Open this file in your browser to view.")
